In [51]:
import numpy as np
import matplotlib.pyplot as plt

import interpolate_matrix as matint

In [9]:
class epsilon:
    def __init__(self, max_linspace, N_LS, N_GL):
        self.linspace_max = max_linspace
        self.N_LS = N_LS
        self.N_GL = N_GL

        x_GL, w_GL = np.polynomial.laguerre.laggauss(N_GL)

        self.eps = np.zeros(N_LS + N_GL)
        self.w = np.zeros_like(self.eps)
        
        self.eps[:N_LS] = np.linspace(0, max_linspace, N_LS)
        self.eps[N_LS:] = x_GL + max_linspace

        self.w[:N_LS] = self.eps[1]
        self.w[0] *= 0.5
        self.w[N_LS-1] *= 0.5
        self.w[N_LS:] = w_GL * np.exp(self.eps[N_LS:]-max_linspace)

In [52]:
def matrix_interp_extrap(x, e_epsilon, P_arrays):
    rho_ee = np.zeros(len(e_epsilon.eps))
    rho_mm = np.zeros_like(rho_ee)

    rho_ee[:] = 0.5 * P_arrays[:,0] * (1 + P_arrays[:,3])
    rho_mm[:] = 0.5 * P_arrays[:,0] * (1 - P_arrays[:,3])

    if x < e_epsilon.linspace_max:
        func = matint.interpolate_log_fifth
    else:
        func = matint.interpolate_log_linear

    fe_temp = func(x, x_array, rho_ee)
    fm_temp = func(x, x_array, rho_mm)

    return np.array([fe_temp + fm_temp, 0, 0, (fe_temp - fm_temp)/(fe_temp + fm_temp + 1e-100)])


In [50]:
e_og = epsilon(10, 201, 5)

In [46]:
e = epsilon(20, 201, 5)

In [47]:
f = np.exp(-e.eps)

In [48]:
np.sum(f * e.w)

1.0008331944757876

In [49]:
ee = np.linspace(0,20,201)
ff = np.exp(-ee)

np.trapz(ff,ee), 1 - np.exp(-10), np.exp(-10)

(1.0008331924146339, 0.9999546000702375, 4.5399929762484854e-05)